🧠 🦷 🦴 👀 👁 👅 👄 💋 🩸

**Автор:** Миша

**Цель:** посмотреть, как работают и как встраиваются в пайплайн разные штуки для работы с несбалансированными данными.

**Библиотеки** : `imblearn`

In [1]:
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

Воспользуемся для примера первым датасетом (German).

In [73]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data',
                 header = None, sep = ' ')

# based on the .doc data description
df.columns = ['cheq_acc', 'dur_t', 'cred_hist', 'purp', 'cred_amt', 'save_acc', 
              'empl_t', 'inst_to_income', 'pers_status', 'guarant_flg',
              'residence_t', 'prop', 'age', 'inst_plan', 'house', 'n_loans',
              'job', 'n_depend', 'tel_flg', 'foreign_flg', 'target']

cat_vals = ['cheq_acc', 'cred_hist', 'purp', 'save_acc', 
            'empl_t', 'pers_status', 'guarant_flg', 'prop', 
            'inst_plan', 'house', 'job', 'tel_flg', 'foreign_flg']
num_vals = ['dur_t', 'cred_amt', 'inst_to_income', 'residence_t', 
            'age', 'n_loans', 'n_depend']

In [74]:
X = df.drop("target", axis=1)
y = df["target"] - 1

Тут возникла следующая проблема: у классов из библиотеки `imblearn` нет методов `fit` и `transform` -- они не встают в наши пайплайны. Для решения этой проблемы можно использовать:

- [пайплайн от `mblearn`](https://stackoverflow.com/questions/50245684/using-smote-with-gridsearchcv-in-scikit-learn)

- `OverSamplingClassifier` на основе кода [отсюда](https://github.com/analyticalmindsltd/smote_variants/issues/8).

Покажем ниже, как работает второй способ:

In [64]:
from sklearn.base import BaseEstimator, ClassifierMixin

class OverasamplingClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, oversampler, classifier):
        self.oversampler= oversampler
        self.classifier= classifier

    def fit(self, X, y=None):
        X_samp, y_samp= self.oversampler.fit_resample(X, y)  # вот тут поменял название метода
        self.classifier.fit(X_samp, y_samp)
        return self

    def predict(self, X):
        return self.classifier.predict(X)

    def predict_proba(self, X):
        return self.classifier.predict_proba(X)

    def get_params(self, deep=True):
        return {'oversampler': self.oversampler, 'classifier': self.classifier}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

Протестируем на следующих алгоритмах:

In [60]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler

# RandomOverSampler

In [75]:
pipeline = Pipeline([
    ('woe', WoEEncoder(variables=cat_vals)),
    ('model', OverasamplingClassifier(
        RandomOverSampler(),
        LogisticRegression(penalty='none', max_iter=1000)
    )
    )
])

pipeline.fit(X, y)

Pipeline(steps=[('woe',
                 WoEEncoder(variables=['cheq_acc', 'cred_hist', 'purp',
                                       'save_acc', 'empl_t', 'pers_status',
                                       'guarant_flg', 'prop', 'inst_plan',
                                       'house', 'job', 'tel_flg',
                                       'foreign_flg'])),
                ('model',
                 OverasamplingClassifier(classifier=LogisticRegression(max_iter=1000,
                                                                       penalty='none'),
                                         oversampler=RandomOverSampler()))])

In [76]:
pipeline.predict(X)[:5]

array([0, 1, 0, 1, 1], dtype=int64)

# RandomUnderSampler

Несмотря на название вспомогательного класса, тоже работает :)

In [78]:
pipeline = Pipeline([
    ('woe', WoEEncoder(variables=cat_vals)),
    ('model', OverasamplingClassifier(
        RandomUnderSampler(),
        LogisticRegression(penalty='none', max_iter=1000)
    )
    )
])

pipeline.fit(X, y)

Pipeline(steps=[('woe',
                 WoEEncoder(variables=['cheq_acc', 'cred_hist', 'purp',
                                       'save_acc', 'empl_t', 'pers_status',
                                       'guarant_flg', 'prop', 'inst_plan',
                                       'house', 'job', 'tel_flg',
                                       'foreign_flg'])),
                ('model',
                 OverasamplingClassifier(classifier=LogisticRegression(max_iter=1000,
                                                                       penalty='none'),
                                         oversampler=RandomUnderSampler()))])

In [79]:
pipeline.predict(X)[:5]

array([0, 1, 0, 1, 1], dtype=int64)

# SMOTE

In [80]:
pipeline = Pipeline([
    ('woe', WoEEncoder(variables=cat_vals)),
    ('model', OverasamplingClassifier(
        SMOTE(k_neighbors=3),
        LogisticRegression(penalty='none', max_iter=1000)
    )
    )
])

pipeline.fit(X, y)

Pipeline(steps=[('woe',
                 WoEEncoder(variables=['cheq_acc', 'cred_hist', 'purp',
                                       'save_acc', 'empl_t', 'pers_status',
                                       'guarant_flg', 'prop', 'inst_plan',
                                       'house', 'job', 'tel_flg',
                                       'foreign_flg'])),
                ('model',
                 OverasamplingClassifier(classifier=LogisticRegression(max_iter=1000,
                                                                       penalty='none'),
                                         oversampler=SMOTE(k_neighbors=3)))])

In [81]:
pipeline.predict(X)[:5]

array([0, 1, 0, 1, 1], dtype=int64)

# ADASYN

In [82]:
pipeline = Pipeline([
    ('woe', WoEEncoder(variables=cat_vals)),
    ('model', OverasamplingClassifier(
        ADASYN(),
        LogisticRegression(penalty='none', max_iter=1000)
    )
    )
])

pipeline.fit(X, y)

Pipeline(steps=[('woe',
                 WoEEncoder(variables=['cheq_acc', 'cred_hist', 'purp',
                                       'save_acc', 'empl_t', 'pers_status',
                                       'guarant_flg', 'prop', 'inst_plan',
                                       'house', 'job', 'tel_flg',
                                       'foreign_flg'])),
                ('model',
                 OverasamplingClassifier(classifier=LogisticRegression(max_iter=1000,
                                                                       penalty='none'),
                                         oversampler=ADASYN()))])

In [83]:
pipeline.predict(X)[:5]

array([0, 1, 0, 1, 1], dtype=int64)

Выводы:

- отчасти удалось решить проблему с тем, что классы `imblearn` не встают в пайплайн
- тем не менее, у такого подхода есть проблема - мы вынуждены делать under / over sampling в самом конце, перед обучением модели, и не можем поставить его куда-то раньше.